In [ ]:
import numpy as np
import pandas as pd
import scipy
import os
from sklearn import metrics
from matplotlib import pyplot as plt
import matplotlib as mpl
import pickle
import seaborn as sns
import matplotlib
%matplotlib inline

from scipy import stats

import seaborn as sns
import matplotlib.pyplot as plt
import matplotlib.ticker as ticker
sns.set(style="whitegrid")


In [ ]:
dset = "2yrprev_within3"

In [ ]:
# NOTE: the categorical ones are NOT the one-hot encoded version for the model, but the raw versions from before standardization

cognitive_features = ['cts_animals', 'cts_bname', 'cts_catflu','cts_db', 'cts_delay', 'cts_df', 'cts_doperf', 'cts_ebdr', 'cts_ebmt',\
            'cts_fruits', 'cts_idea', 'cts_lopair', 'cts_mmse30', 'cts_nccrtd','cts_pmat', 'cts_pmsub', 'cts_read_nart', \
            'cts_sdmt', 'cts_story', 'cts_stroop_cname', 'cts_stroop_wread', 'cts_wli', 'cts_wlii', 'cts_wliii']
medical_features_sums = ['med_con_sum_cum', 'vasc_3dis_sum', 'vasc_risks_sum']
continuous_demographics = ['age_at_visit', 'educ']

composite_vars = {
    "cogn_ep": ["cts_wli", "cts_wlii", "cts_wliii", "cts_ebmt", "cts_ebdr",  "cts_story","cts_delay"],
    "cogn_po": ["cts_lopair", "cts_pmat"],
    "cogn_ps": ["cts_sdmt", "cts_nccrtd", "cts_stroop_cname", "cts_stroop_wread"],
    "cogn_se":  ["cts_bname", "cts_catflu", "cts_read_nart"],
    "cogn_wo": ["cts_db", "cts_df", "cts_doperf"],
    "cogn_global":  ["cts_wli", "cts_wlii", "cts_wliii", "cts_ebmt", "cts_ebdr",  "cts_story","cts_delay",\
                     "cts_lopair", "cts_pmat", "cts_sdmt", "cts_nccrtd", "cts_stroop_cname", "cts_stroop_wread",
                     "cts_bname", "cts_catflu", "cts_read_nart", "cts_db", "cts_df", "cts_doperf"] }
    
# these can stay as is
binary = ['hypertension_cum', 'cancer_cum','diabetes_sr_rx', 'dm_cum', 'headinjrloc_cum', 'lostcons',\
                         'thyroid_cum', 'chf_cum', 'claudication_cum', 'heart_cum', 'stroke_cum', "msex", "spanish"]


# these need to be one hot encoded
categorical = ['apoe_4count', 'race', 'dcfdx']


In [ ]:
load_data = pd.read_csv("../DATA/PROCESSED/standardized/merged_data_all_%s.csv"%dset, index_col=0)

sample_info = ["projid","study","fu_year","scaled_to", "onset_label_time", 'onset_label_time_binary']
data = load_data[sample_info]

In [ ]:
feature_names = np.setdiff1d(load_data.columns, sample_info)
features = load_data[feature_names]

# demographics table

In [ ]:
orig_data_features = pd.read_csv("../DATA/PROCESSED/merged_kept_data_2yrprev_within3.csv").drop(['Unnamed: 0'], axis=1)
orig_data_features["apoe_4count"] = orig_data_features["apoe_genotype"].apply(lambda x: 0 if x in [22., 23., 33.] else 1 if x in [24., 34.] else 2 if x == 44. else np.nan)


In [ ]:
categorical_features =  binary + categorical 
continuous_features = cognitive_features + continuous_demographics + medical_features_sums + list(composite_vars.keys())

demo_feats = ['age_at_visit', 'msex','educ', 'race', 'spanish', 'apoe_4count']
# cog_feats = ['dcfdx', 'cogn_ep', 'cogn_po', 'cogn_ps', 'cogn_se', 'cogn_wo','cogn_global']
cog_feats = ['dcfdx', 'cogn_global']

med_feats = ['med_con_sum_cum', 'vasc_3dis_sum', 'vasc_risks_sum', 'cancer_cum','claudication_cum', \
        'diabetes_sr_rx', 'dm_cum', 'headinjrloc_cum', 'heart_cum', 'hypertension_cum','stroke_cum', 'thyroid_cum'] 

In [ ]:
t_test_vars = ['age_at_visit', 'educ', 'cogn_ep', 'cogn_po', 'cogn_ps', 'cogn_se', 'cogn_wo','cogn_global']

t_test_vars += ['med_con_sum_cum','vasc_3dis_sum', 'vasc_risks_sum']
#u_test_vars = ['med_con_sum_cum','vasc_3dis_sum', 'vasc_risks_sum']
    
chi_test_vars = ['msex', 'race', 'spanish', 'apoe_4count', 'dcfdx', 'cancer_cum','claudication_cum', \
        'diabetes_sr_rx', 'dm_cum', 'headinjrloc_cum', 'heart_cum', 'hypertension_cum','stroke_cum', 'thyroid_cum'] 

In [ ]:
percentages=True

for feat_group in [demo_feats, cog_feats, med_feats]:

    for feat in feat_group:
        controls = orig_data_features[data["onset_label_time_binary"]==0][feat]
        dementias = orig_data_features[data["onset_label_time_binary"]==1][feat]

        if feat in t_test_vars:


            t,p = stats.ttest_ind(controls.dropna().values,dementias.dropna().values)
            p_stars = "***" if p<.001 else "**" if p <.01 else "*" if p < .05 else ""

            print("%s & $t=%.2f^{%s}$  & $%.2f \pm %.2f$ &  $%.2f \pm %.2f$  \\\\ "%(feat.replace("_", "\_"),t,p_stars, controls.mean(), controls.std(), dementias.mean(), dementias.std()))

        else:

            # create dictionary of counts for observed values of feature
            control_valcounts = {}
            for i,v in enumerate(controls.value_counts().index):
                control_valcounts[v] = controls.value_counts().values[i]
            dem_valcounts = {}
            for i,v in enumerate(dementias.value_counts().index):
                dem_valcounts[v] = dementias.value_counts().values[i]

            #get union of all values seen (just in case one of the groups has some 0s for some values)
            all_vals = np.union1d(list(dem_valcounts.keys()), list(control_valcounts.keys()))
            
            # generate contingency table (shape: values observed x groups)
            contingency_table = np.array([[control_valcounts[elt], dem_valcounts[elt]] for elt in all_vals])


            chi2_stat, p, dof, ex = stats.chi2_contingency(contingency_table)
            p_stars = "***" if p<.001 else "**" if p <.01 else "*" if p < .05 else ""

            
            
            if percentages:
                if len(all_vals) < 3:
                    controlfrac =control_valcounts[1]/np.sum(list(control_valcounts.values())) * 100
                    demfrac =dem_valcounts[1]/np.sum(list(dem_valcounts.values())) * 100
                    
                    print("%s & $\chi^2=%.2f^{%s}$  & $%.1f\%%$ &  $%.1f\%%$  \\\\ "%(feat.replace("_", "\_"), chi2_stat,p_stars,controlfrac, demfrac))

                
                else:
                    outcomes_str = "/".join(all_vals.astype(int).astype(str))
                    
                    controlvals = (np.round(contingency_table[:,0]/sum(contingency_table[:,0])*100,1))
                    demvals = (np.round(contingency_table[:,1]/sum(contingency_table[:,1])*100,1))
                    controlvals_str = "/".join(["%s\%%"%x for x in controlvals.astype(str)])
                    demvals_str = "/".join(["%s\%%"%x for x in demvals.astype(str)])
                
                    print("%s ($%s$) & $\chi^2=%.2f^{%s}$  & $%s$ &  $%s$  \\\\ "%(feat.replace("_", "\_"),outcomes_str, chi2_stat,p_stars,controlvals_str, demvals_str))
            else:
                outcomes_str = "/".join(all_vals.astype(int).astype(str))
                controlvals_str = "/".join(contingency_table[:,0].astype(int).astype(str))
                demvals_str = "/".join(contingency_table[:,1].astype(int).astype(str))
                
                print("%s ($%s$) & $\chi^2=%.2f^{%s}$  & $%s$ &  $%s$  \\\\ "%(feat.replace("_", "\_"),outcomes_str, chi2_stat,p_stars,controlvals_str, demvals_str))
    print("\\\\")